In [2]:
import pandas as pd 


df= pd.read_excel(r"C:\Users\kfq6\Documents\Data\Sammedag_master_HbA1c_Features.xlsx")

In [7]:
# Sort by patient and anchor_date
df = df.sort_values(by=["DW_EK_Borger", "anchor_date"])

# Define future target: next available HbA1c per patient
df["HbA1c_Target"] = df.groupby("DW_EK_Borger")["LABmean__hb_b_haemoglobin_a1c_ifcc_mmol_mol"].shift(-1)


In [8]:
df.head(10)

,DW_EK_Borger,age,comp_eye,comp_renal,comp_neuro,comp_periph,comp_foot,comp_multiple,comp_uns,anchor_date,...,LDL_prev_year,HbA1c_slope_prev_year,HbA1c_CV_prev_year,HbA1c_MAC_prev_year,diagnosis_Ikke specificeret diabetes,diagnosis_Type 1-diabetes,diagnosis_Type 2-diabetes,sex_M,HbA1c_t_plus_1,HbA1c_Target
0,7371,56.8,0,0,1,0,0,0,0,2022-11-30,...,NaN,NaN,NaN,NaN,False,True,False,True,71.0,71.0
1,7371,56.8,0,0,1,0,0,0,0,2024-02-13,...,2.5,NaN,NaN,NaN,False,True,False,True,71.0,71.0
2,7371,56.8,0,0,1,0,0,0,0,2025-03-05,...,2.4,NaN,NaN,NaN,False,True,False,True,NaN,NaN
3,8395,33.8,0,0,0,0,0,0,0,2022-03-31,...,NaN,NaN,NaN,NaN,False,True,False,True,51.0,51.0
4,8395,33.8,0,0,0,0,0,0,0,2023-08-09,...,2.6,NaN,NaN,NaN,False,True,False,True,52.0,52.0
5,8395,33.8,0,0,0,0,0,0,0,2024-08-07,...,3.9,NaN,NaN,NaN,False,True,False,True,NaN,NaN
6,10562,74.7,0,0,0,0,0,0,1,2022-11-15,...,NaN,NaN,NaN,NaN,False,True,False,True,63.0,63.0
7,10562,74.7,0,0,0,0,0,0,1,2023-10-09,...,2.0,-0.000030,0.032696,3.5,False,True,False,True,69.0,69.0
8,10562,74.7,0,0,0,0,0,0,1,2024-12-02,...,2.2,0.011292,0.047042,3.5,False,True,False,True,NaN,NaN
9,10834,67.7,0,0,0,0,0,0,0,2022-08-31,...,NaN,NaN,NaN,NaN,False,False,True,True,65.0,65.0
